In [27]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import pandas
import math
import os
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.utils import plot_model
from biosppy.signals import ecg
# fix random seed for reproducibility
np.random.seed(7)

In [2]:
DATA_TRAIN_PATH = 'blood-pressure-estimation-data/data_train'
DATA_TRAIN_ANS_PATH = 'blood-pressure-estimation-data/data_train.ans'
DATA_TEST1_PATH = 'blood-pressure-estimation-data/data_test1_blank'
DATA_TEST2_PATH = 'blood-pressure-estimation-data/data_test2_blank'

In [306]:
def load_answers(path):
    sbp_dbp = {}
    with open(path, 'r') as f:
        for line in f:
            vals = line[:-1].split(',')
            sbp_dbp[vals[0]] = (float(vals[1]), float(vals[2]))
    return sbp_dbp

sbp_dbp = load_answers(DATA_TRAIN_ANS_PATH)

def read_ppg_ecg(filename):
    ppg = []
    ecg = []
    with open(filename, 'r') as f:
        for id, line in enumerate(f):
            vals = line[:-1].split(',')
            if id == 0:
                sbp = float(vals[0])
                dbp = float(vals[1])
                continue
            ppg.append(float(vals[0]))
            ecg.append(float(vals[1]))
    return ppg[2000:], ecg[2000:], sbp, dbp

In [620]:
def prepareTrain():
    files = list(sbp_dbp.keys())
    X = []
    Y = []
    cnt = 0
    for file in files:
        try:
            ppg_l, ecg_l, _, _ = read_ppg_ecg(DATA_TRAIN_PATH + '/' + file)
            out_ecg = ecg.ecg(signal=np.array(ecg_l, dtype=float), sampling_rate=500., show=False)
            out_ppg = ecg.ecg(signal=np.array(ppg_l, dtype=float), sampling_rate=500., show=False)
            
            X.append(ppg_l[:10000]+ ecg_l[:10000])
            Y.append([sbp_dbp[file][0], sbp_dbp[file][1]])
            '''
            ppg_peaks = out_ppg['rpeaks']
            ecg_peaks = out_ecg['rpeaks']
            
            if abs(len(ppg_peaks) - len(ecg_peaks)) < 1:
                min_len = min(len(ppg_peaks), len(ecg_peaks))
                dp = ppg_peaks[:min_len] - ecg_peaks[:min_len]
                hr = out_ecg['heart_rate']
                X.append([np.average(np.abs(dp)), np.max(np.abs(dp)), np.min(np.abs(dp)), np.average(hr), np.max(hr), np.min(hr)])
                Y.append([sbp_dbp[file][0], sbp_dbp[file][1]])
            '''
        except Exception as v:
            print(v)
            #break
    return X, Y

In [621]:
X, Y = prepareTrain()
y = np.array(Y)

Not enough beats to compute heart rate.
Not enough beats to compute heart rate.
Not enough beats to compute heart rate.


In [594]:
len(X)

891

In [649]:
learn_size = 800
X_train, X_test = X[:learn_size], X[learn_size:]
Y_train, Y_test = y[:learn_size], y[learn_size:]

In [643]:
len(X_test)

391

In [661]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

clf = RandomForestRegressor(max_depth=2, random_state=17, n_estimators=10, n_jobs=-1)
clf.fit(X_train, Y_train[:, 1])
mean_squared_error(Y_test[:,1], list(map(round, clf.predict(X_test))))

105.45026178010471

In [667]:
clf2 = RandomForestRegressor(max_depth=20, random_state=42, n_estimators=20, n_jobs=8)
clf2.fit(X_train, Y_train[:, 0])
mean_squared_error(Y_test[:,0], list(map(round, clf2.predict(X_test)))) 

451.3979057591623

In [654]:
def prepare_test():
    files = os.listdir(DATA_TEST1_PATH)
    X = []
    cnt = 0
    filled_tests = []
    prepeared_tests = []
    for file in files:
        ppg_l, ecg_l, sbp, dbp = read_ppg_ecg(DATA_TEST1_PATH + '/' + file)
        
        if (sbp == 0 and dbp == 0):
            out_ecg = ecg.ecg(signal=np.array(ecg_l, dtype=float), sampling_rate=500., show=False)
            out_ppg = ecg.ecg(signal=np.array(ppg_l, dtype=float), sampling_rate=500., show=False)
            X.append(ppg_l[:10000]+ ecg_l[:10000])
            '''
            ppg_peaks = out_ppg['rpeaks']
            ecg_peaks = out_ecg['rpeaks']
            #if abs(len(ppg_peaks) - len(ecg_peaks)) < 5:
            min_len = min(len(ppg_peaks), len(ecg_peaks))
            dp = ppg_peaks[:min_len] - ecg_peaks[:min_len]
            hr = out_ecg['heart_rate']
            X.append([np.average(np.abs(dp)), np.max(np.abs(dp)), np.min(np.abs(dp)), np.average(hr), np.max(hr), np.min(hr)])
            '''
            prepeared_tests.append(file)
        else:
            filled_tests.append((file, sbp, dbp))
    return X, filled_tests, prepeared_tests

In [694]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
def baseline_model():
    model = Sequential()
    model.add(Dense(10, input_dim=20000, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer='adam')
    return model

In [692]:
model = {}

In [ ]:
model = baseline_model()
model.fit(np.array(X_train).reshape((len(X_train), 20000)), Y_train[:,0], epochs=20, batch_size=20, verbose = 1)

Epoch 1/20


In [675]:
mean_squared_error(list(map(round, model.predict(np.array(X_test).reshape((len(X_test),20000 )))[:,0])), Y_test[:,0])

9347.167539267015

In [588]:
model2 = baseline_model()
model2.fit(np.array(X_train).reshape((len(X_train), 6)), Y_train[:,1], epochs=100, batch_size=50, verbose = 0)

In [589]:
mean_squared_error(list(map(round, model2.predict(np.array(X_test).reshape((len(X_test), 6)))[:,0])), Y_test[:,1]) / len(X_test)

4.2374010828821325

In [655]:
X, ft, pt = prepare_test()

In [668]:
sbp = list(map(round, clf2.predict(X)))
dbp = list(map(round, clf.predict(X)))

#sbp = list(map(round, model.predict(np.array(X).reshape((len(X), 6)))[:,0]))
#dbp = list(map(round, model2.predict(np.array(X).reshape((len(X), 6)))[:,0]))

for v in ft:
    print(v[0], int(v[1]), int(v[2]), sep = ',')
for i, v in enumerate(pt):
    print(v, int(sbp[i]), int(dbp[i]), sep = ',')

subj29log1897.csv,95,61
subj13log802.csv,137,75
subj29log1895.csv,91,61
subj12log731.csv,107,80
subj13log796.csv,137,67
subj12log729.csv,106,72
subj29log1903.csv,102,56
subj29log1906.csv,93,62
subj12log737.csv,109,81
subj8log461.csv,106,64
subj12log736.csv,113,78
subj15log939.csv,106,76
subj29log1893.csv,100,55
subj15log928.csv,106,62
subj8log466.csv,122,62
subj12log733.csv,102,77
subj15log941.csv,118,84
subj13log799.csv,142,73
subj13log804.csv,133,71
subj19log1189.csv,133,91
subj15log940.csv,100,71
subj13log803.csv,144,75
subj13log801.csv,126,59
subj12log740.csv,120,79
subj19log1187.csv,122,79
subj19log1190.csv,125,77
subj29log1892.csv,88,50
subj8log462.csv,109,61
subj12log732.csv,120,79
subj15log938.csv,106,72
subj8log456.csv,102,55
subj15log931.csv,106,73
subj13log795.csv,131,70
subj13log800.csv,137,71
subj29log1900.csv,86,54
subj15log932.csv,109,77
subj13log797.csv,143,79
subj15log927.csv,104,75
subj29log1901.csv,101,59
subj8log464.csv,118,62
subj15log930.csv,107,77
subj8log460.csv